In [ ]:
from D_oblique_decision_trees.evaluation.benchmark_runner import DepthSweepRunner
from src.load_shapes import load_all_shape_datasets

all_data = load_all_shape_datasets()

In [ ]:
datasets_2d_dict = {
  "barbell_2d": all_data["barbell_2d"],
  "sine_wave_2d": all_data["sine_wave_2d"],
  "star_2d": all_data["star_2d"],
  "radial_segment_2d": all_data["radial_segment_2d"],
  "rectangle_2d": all_data["rectangle_2d"]
}

# datasets_3d_dict = {
#   "barbell_3d": all_data["barbell_3d"],
#   "radial_segment_3d": all_data["radial_segment_3d"],
#   "saddle_3d": all_data["saddle_3d"]
# }

runner = DepthSweepRunner(datasets_2d_dict, max_depth=7)
df = runner.run()

df = runner.reorder_and_print(df)
runner.plot_results(df, metric="accuracy")            # separate lines per dataset
runner.plot_aggregated_metric(df, metric="accuracy")  # single line per algorithm, averaged across datasets
runner.save_results(df, "my_depth_sweep.csv")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from src.load_shapes import load_all_shape_datasets

from D_oblique_decision_trees.converters.dispatcher import convert_tree
from D_oblique_decision_trees.visualisation import plot_decision_boundaries, print_tree_structure

from Ensembles_of_Oblique_Decision_Trees.Decision_trees.HouseHolder_CART import HHCartClassifier
from Ensembles_of_Oblique_Decision_Trees.Decision_trees.RandCART import RandCARTClassifier
from Ensembles_of_Oblique_Decision_Trees.Decision_trees.CO2 import CO2Classifier
from Ensembles_of_Oblique_Decision_Trees.Decision_trees.Oblique_Classifier_1 import ObliqueClassifier1
from Ensembles_of_Oblique_Decision_Trees.Decision_trees.WODT import WeightedObliqueDecisionTreeClassifier
from Ensembles_of_Oblique_Decision_Trees.Decision_trees.segmentor import MeanSegmentor
from Ensembles_of_Oblique_Decision_Trees.Decision_trees.split_criteria import gini

In [ ]:
# Load & scale dataset
datasets = load_all_shape_datasets()
X, y = datasets["barbell_2d"]

In [ ]:
model_registry = {
    "hhcart": lambda: HHCartClassifier(impurity=gini, segmentor=MeanSegmentor(), max_depth=7),
    "randcart": lambda: RandCARTClassifier(impurity=gini, segmentor=MeanSegmentor(), max_depth=7),
    "oc1": lambda: ObliqueClassifier1(max_depth=7, min_samples_split=2),
    "wodt": lambda: WeightedObliqueDecisionTreeClassifier(max_depth=7, min_samples_split=2, max_features='all')
}

In [ ]:
# model_registry = {"co2": lambda: CO2Classifier(impurity=gini, segmentor=MeanSegmentor(), max_depth=8)}

In [ ]:
ncols = 2
nrows = (len(model_registry) + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 5 * nrows))
axes = axes.flatten()

for i, (model_type, constructor) in enumerate(model_registry.items()):
    model = constructor()
    model.fit(X, y)

    tree = convert_tree(model, model_type=model_type)
    ax = axes[i]
    print_tree_structure(tree)
    plot_decision_boundaries(tree, X, y, ax=ax, xlim=(0, 1), ylim=(0, 1))
    # plot_decision_boundaries(tree, X, y, ax=ax, xlim=(-3, 3), ylim=(-3, 3))
    ax.set_title(f"{model_type.upper()}")

for j in range(i + 1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
from evaluation.evaluator import evaluate_tree
import pandas as pd
import time

# Filter to 2D datasets only
datasets_2d = {name: (X, y) for name, (X, y) in datasets.items() if X.shape[1] == 3}

results = []

for dataset_name, (X, y) in datasets_2d.items():
    for model_name, constructor in model_registry.items():
        print(f"Running {model_name} on {dataset_name}...")
        model = constructor()

        start = time.time()
        model.fit(X, y)
        end = time.time()

        tree = convert_tree(model, model_type=model_name)

        metrics = evaluate_tree(tree, X, y, training_time=end - start)
        metrics["model"] = model_name
        metrics["dataset"] = dataset_name
        results.append(metrics)

df_results = pd.DataFrame(results)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(data=df_results, x="dataset", y="accuracy", hue="model")
plt.xticks(rotation=45)
plt.title("Model Accuracy per Dataset")
plt.tight_layout()
plt.show()